<br><br>
# **SQLalchemy**
<br>
**DB**를 **Pandas**로 중계하며 Create, Read, Update, Delete 작업하기 <br>
**[SQLITE3](https://www.haruair.com/blog/1682) &nbsp;
[sqlalchemy_tutorial](http://docs.sqlalchemy.org/en/latest/core/tutorial.html) &nbsp;
[sqlalchemy_Example](http://docs.sqlalchemy.org/en/latest/orm/examples.html) &nbsp;
[MARIADB](https://swalloow.github.io/db-to-dataframe) &nbsp;
[MARIADB](http://agfree.cafe24.com/?p=1477)**
1. 지금까지 Sqlite3를 사용했지만 용량이 넘치기 시작했다
1. 간단한 SQL 문법은 Sqlite3 로 익히더라고
1. 이제부터는 대부분의 내용들을 **MariaDB** 까지도 포함해서 저장 및 활용하기

In [1]:
# Psql 에 대해서 익히려고 했지만, odroid Xu4 등에서 설치가 안됨
# 범용적인 활용을 위해선 우선은 MariaDB를 익혀서 작업을 하고
# 추후에 AWS등을 활용하여 PostgreSQL를 설치가능하면 확장해 나아가기

# 아래의 3개 모듈을 활용한다
import pymysql       # MySQL cursor 연결
import pandas as pd  # 결과물 출력 및 생성

<br>
## **1 engine : DB 접속**
**Driver** &nbsp; :// &nbsp;  **filename** &nbsp;&nbsp; ex) sqlite:///:memory:
1. DB별 다른 문법들을 일원화 해서 작업 가능하도록 돕는다
1. 한가지 DB를 익히고 아면 Connector만 변경시, 다른 DB에도 적용이 가능
1. Django 내부에서도 이를 활용하는 듯 하다


In [2]:
# Python 에서 DB를 쉽게 연결하도록 중계
import sqlalchemy    
sqlalchemy.__version__

'1.2.6'

In [3]:
# SQLITE3 에 접속하기
# echo는 로그를 위한 플래그로 순수 SQL 코드를 보여준다.
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

# SQLITE3 엔진을 실행 (실행시 최초 접속)
engine.execute("select 1").scalar()
# 현재는 생성된 테이블이 없다
engine.table_names() 

2018-06-21 17:53:04,171 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-06-21 17:53:04,172 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,174 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-06-21 17:53:04,175 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,177 INFO sqlalchemy.engine.base.Engine select 1
2018-06-21 17:53:04,178 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,181 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-06-21 17:53:04,181 INFO sqlalchemy.engine.base.Engine ()


[]

<br>
## **2 Mapping Class : Connection 객체**
https://www.haruair.com/blog/1682

<br>
### **01 Mapping Class  정의하기**
https://www.haruair.com/blog/1682

In [4]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String
# 사용자 접속 Cursor 를 Class로 정의한다
class User(Base):
    # 테이블 이름
    __tablename__ = 'users'  

    # 테이블의 컬럼들 정의
    id       = Column(Integer, primary_key=True)
    name     = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __init__(self, name, fullname, password):
        self.name     = name
        self.fullname = fullname
        self.password = password

    def __repr__(self):
        return "<User('%s', '%s', '%s')>" % (self.name, self.fullname, self.password)

In [5]:
# 위에서 정의된 DATABASE 의 메타정보를 출력한다
Base.metadata.create_all(engine)

2018-06-21 17:53:04,255 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-06-21 17:53:04,256 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,259 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2018-06-21 17:53:04,260 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,262 INFO sqlalchemy.engine.base.Engine COMMIT


<br>
### **02 Mapping Class : 상세한 묘사**
매핑 클래스 중복시 충돌을 피한다 [StockFlow](https://stackoverflow.com/questions/43614981/sqlalchemy-problems-with-inherited-classes-in-the-same-table)

In [6]:
Column(String(50))

Column(None, String(length=50), table=None)

In [7]:
# PK를 생성시 Sequence를 필요로 하는경우 정의에 추가한다
from sqlalchemy import Sequence
Column(Integer, Sequence('user_id_seq'), primary_key=True)

Column(None, Integer(), table=None, primary_key=True, nullable=False, default=Sequence('user_id_seq'))

In [8]:
# Sequence 추가 (몇몇 DB에 대해서 적용된다)
# User 매핑 클래스
class User(Base):

    __tablename__  = 'users'
    __table_args__ = {'extend_existing': True}  # 같은 이름의 User가 존재시 이를 덮어쓴다
    
    id       = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name     = Column(String(50))
    fullname = Column(String(50))
    password = Column(String(12))

    def __init__(self, name, fullname, password):
        self.name     = name
        self.fullname = fullname
        self.password = password

    def __repr__(self):
        return "<User('%s', '%s', '%s')>" % (self.name, self.fullname, self.password)

/home/markbaum/Python/django/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  item.__name__


<br>
### **03 Instance : 매핑클래스의 활용**
__init__() 에 별도 정의를 하지 않아도 자동으로 매핑된다

In [9]:
ed_user = User('haruair', 'Edward Kim', '1234')
print(".name : {} \n.password : {} \n.id : {}".\
      format(ed_user.name, ed_user.password, ed_user.id))

.name : haruair 
.password : 1234 
.id : None


<br>
## **3 Session**
ORM은 데이터베이스를 session을 이용해 다룬다 

<br>
### **01 Session 만들기**
Instance : 매핑클래스의 활용

In [10]:
from sqlalchemy.orm import sessionmaker

# Session을 생성한 뒤 위에서 정의한 engine을 추가
Session = sessionmaker(bind = engine)
session = Session()

<br>
### **02 Session 을 활용한 객체수정**
session.add() , session.add_all()
1. ORM은 데이터베이스를 session을 활용
1. ed_user : 객체
1. our_user : 객체 Instance 

In [11]:
ed_user= User('haruair', 'Edward Kim', '1234')
session.add(ed_user)

# <User('haruair', 'Edward Kim', 'secret')>
our_user = session.query(User).filter_by(name='haruair').first()
our_user     

2018-06-21 17:53:04,329 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-21 17:53:04,331 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-06-21 17:53:04,332 INFO sqlalchemy.engine.base.Engine ('haruair', 'Edward Kim', '1234')
2018-06-21 17:53:04,334 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2018-06-21 17:53:04,335 INFO sqlalchemy.engine.base.Engine ('haruair', 1, 0)


<User('haruair', 'Edward Kim', '1234')>

In [12]:
# our_user 객체 인스턴스가 Map 구조임을 확인한다
# ORM의 컨셉이 identity map이라서 
# session에서 하는 모든 처리들이 실제 데이터셋과 함께 동작한다
ed_user is our_user     # True

True

In [13]:
# Session 에 Padding 된 내용들 확인
session.add_all([
    User('wendy', 'Wendy Williams', 'foobar'),
    User('mary', 'Mary Contrary', 'xxg527'),
    User('fred', 'Fred Flinstone', 'blar')])

ed_user.password = 'test1234'
session.dirty        

IdentitySet([<User('haruair', 'Edward Kim', 'test1234')>])

In [14]:
# 새롭게 추가한 내용 확인
list(session.new)

[<User('wendy', 'Wendy Williams', 'foobar')>,
 <User('mary', 'Mary Contrary', 'xxg527')>,
 <User('fred', 'Fred Flinstone', 'blar')>]

In [15]:
# .commit()을 실행시 추가 이력을 반영
session.commit()

2018-06-21 17:53:04,361 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2018-06-21 17:53:04,362 INFO sqlalchemy.engine.base.Engine ('test1234', 1)
2018-06-21 17:53:04,363 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-06-21 17:53:04,364 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2018-06-21 17:53:04,366 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-06-21 17:53:04,366 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2018-06-21 17:53:04,367 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-06-21 17:53:04,368 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blar')
2018-06-21 17:53:04,369 INFO sqlalchemy.engine.base.Engine COMMIT


<br>
### **03 Roll-back**
후진 복귀의 의미로, 업데이트 이전 저널 파일을 사용하여 이전 상태로 되돌린다

In [16]:
# ed_user 정보를 확인한다 
ed_user.id

2018-06-21 17:53:04,376 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-21 17:53:04,377 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2018-06-21 17:53:04,378 INFO sqlalchemy.engine.base.Engine (1,)


1

In [17]:
# 가짜 계정을 생성한다
ed_user.name = 'edkim'

fake_user = User('fakeuser', 'Invalid', '12345')
session.add(fake_user)
session.query(User).filter(User.name.in_(['edkim', 'fakeuser'])).all()

2018-06-21 17:53:04,388 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2018-06-21 17:53:04,389 INFO sqlalchemy.engine.base.Engine ('edkim', 1)
2018-06-21 17:53:04,390 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2018-06-21 17:53:04,391 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2018-06-21 17:53:04,392 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2018-06-21 17:53:04,393 INFO sqlalchemy.engine.base.Engine ('edkim', 'fakeuser')


[<User('edkim', 'Edward Kim', 'test1234')>,
 <User('fakeuser', 'Invalid', '12345')>]

In [18]:
session.rollback()
ed_user.name            # 'haruair'

2018-06-21 17:53:04,400 INFO sqlalchemy.engine.base.Engine ROLLBACK
2018-06-21 17:53:04,402 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-06-21 17:53:04,403 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2018-06-21 17:53:04,404 INFO sqlalchemy.engine.base.Engine (1,)


'haruair'

In [19]:
fake_user in session    # False

False

<br>
## **4 Query & Filter**
session에서 query() 메소드로 생성

<br>
### **01 Query**
session에서 query() 메소드로 생성

In [20]:
# 위에서 User 인스턴스를 부를 때 Query를 이미 써봤다
# iterative context를 evaluated할 때, User 객체 리스트를 반환한다

for instance in session.query(User).order_by(User.id):
    print (instance.name, instance.fullname)

2018-06-21 17:53:04,419 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2018-06-21 17:53:04,421 INFO sqlalchemy.engine.base.Engine ()
haruair Edward Kim
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


In [21]:
# Query는 KeyedTuple 클래스 통해 튜플로 반환
# 각 저장된 값들은 클래스 이름이나 속성 이름과 동일하다.

for row in session.query(User, User.name).all():
    print (row.User, row.name)

2018-06-21 17:53:04,428 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2018-06-21 17:53:04,429 INFO sqlalchemy.engine.base.Engine ()
<User('haruair', 'Edward Kim', 'test1234')> haruair
<User('wendy', 'Wendy Williams', 'foobar')> wendy
<User('mary', 'Mary Contrary', 'xxg527')> mary
<User('fred', 'Fred Flinstone', 'blar')> fred


In [22]:
# label()로 컬럼 이름을 다르게 출력

for row in session.query(User.name.label('name_label')).all():
    print (row.name_label)

2018-06-21 17:53:04,437 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2018-06-21 17:53:04,438 INFO sqlalchemy.engine.base.Engine ()
haruair
wendy
mary
fred


In [23]:
# User 클래스 엔티티는 aliased를 이용해 제어한다

from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')

for row in session.query(user_alias, user_alias.name).all():
    print (row.user_alias)

2018-06-21 17:53:04,451 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias
2018-06-21 17:53:04,452 INFO sqlalchemy.engine.base.Engine ()
<User('haruair', 'Edward Kim', 'test1234')>
<User('wendy', 'Wendy Williams', 'foobar')>
<User('mary', 'Mary Contrary', 'xxg527')>
<User('fred', 'Fred Flinstone', 'blar')>


In [24]:
# LIMIT이나 OFFSET을 포함한 기본적인 Query 동작
# order by와 함께 [:slice] 를 활용한다
for user in session.query(User).order_by(User.id)[1:3]:
    print (user)

2018-06-21 17:53:04,461 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2018-06-21 17:53:04,461 INFO sqlalchemy.engine.base.Engine (2, 1)
<User('wendy', 'Wendy Williams', 'foobar')>
<User('mary', 'Mary Contrary', 'xxg527')>


In [25]:
# filter_by() :  filter
for name in session.query(User.name).filter_by(fullname='Edward Kim'):
    print (name)

2018-06-21 17:53:04,469 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2018-06-21 17:53:04,471 INFO sqlalchemy.engine.base.Engine ('Edward Kim',)
('haruair',)


In [26]:
# filter() : 좀더 유연한 SQL 표현이 가능
# 매핑클래스에서 사용한 클래스 단위의 속성과 파이썬 표준 연산자를 쓸 수 있다.
for name in session.query(User.name).filter(User.fullname=='Edward Kim'):
    print (name)

2018-06-21 17:53:04,478 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2018-06-21 17:53:04,479 INFO sqlalchemy.engine.base.Engine ('Edward Kim',)
('haruair',)


In [27]:
# query() 체인 방식으로도 활용 가능하다
for name in session.query(User).\
            filter(User.name=='haruair').\
            filter(User.fullname=='Edward Kim'):
    print (user)

2018-06-21 17:53:04,486 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2018-06-21 17:53:04,488 INFO sqlalchemy.engine.base.Engine ('haruair', 'Edward Kim')
<User('mary', 'Mary Contrary', 'xxg527')>


<br>
### **02 Filter**
일반 필터(filter) 연산자들

In [28]:
# Equal 조건 : .filter(User.name == 'ed') : 
session.query(User).filter(User.name == 'ed').all()

2018-06-21 17:53:04,497 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2018-06-21 17:53:04,498 INFO sqlalchemy.engine.base.Engine ('ed',)


[]

In [29]:
# Not Equal 조건 : .filter(User.name != 'ed')
session.query(User).filter(User.name != 'ed').all()

2018-06-21 17:53:04,507 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name != ?
2018-06-21 17:53:04,508 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User('haruair', 'Edward Kim', 'test1234')>,
 <User('wendy', 'Wendy Williams', 'foobar')>,
 <User('mary', 'Mary Contrary', 'xxg527')>,
 <User('fred', 'Fred Flinstone', 'blar')>]

In [30]:
# LIKE 조건 : .filter(User.name.like('%ed%'))
session.query(User).filter(User.name.like('%ed%')).all()

2018-06-21 17:53:04,516 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?
2018-06-21 17:53:04,517 INFO sqlalchemy.engine.base.Engine ('%ed%',)


[<User('fred', 'Fred Flinstone', 'blar')>]

In [31]:
# IN_ 조건 : .filter(User.name.in_(['ed', 'wendy', 'jack']))
session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])).all()

2018-06-21 17:53:04,525 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?, ?)
2018-06-21 17:53:04,527 INFO sqlalchemy.engine.base.Engine ('ed', 'wendy', 'jack')


[<User('wendy', 'Wendy Williams', 'foobar')>]

In [32]:
# 서브쿼리
# query.filter(User.name.in_(session.query().filter(User.name.like())))
session.query(User).filter(
    User.name.in_(
        session.query(User.name).filter(
            User.name.like('%ed%')))).all()

2018-06-21 17:53:04,538 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (SELECT users.name AS users_name 
FROM users 
WHERE users.name LIKE ?)
2018-06-21 17:53:04,540 INFO sqlalchemy.engine.base.Engine ('%ed%',)


[<User('fred', 'Fred Flinstone', 'blar')>]

In [33]:
# ~ NOT IN
# query.filter(~User.name.in_(['ed', 'wendy', 'jack'])) 
session.query(User).filter(~User.name.in_(['ed', 'wendy', 'jack'])).all()

2018-06-21 17:53:04,550 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name NOT IN (?, ?, ?)
2018-06-21 17:53:04,552 INFO sqlalchemy.engine.base.Engine ('ed', 'wendy', 'jack')


[<User('haruair', 'Edward Kim', 'test1234')>,
 <User('mary', 'Mary Contrary', 'xxg527')>,
 <User('fred', 'Fred Flinstone', 'blar')>]

In [34]:
# IS NULL : .filter(User.name == None)
session.query(User).filter(User.name == None).all()

2018-06-21 17:53:04,560 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IS NULL
2018-06-21 17:53:04,561 INFO sqlalchemy.engine.base.Engine ()


[]

In [35]:
# IS NOT NULL : .filter(User.name == None)
session.query(User).filter(User.name != None).all()

2018-06-21 17:53:04,568 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IS NOT NULL
2018-06-21 17:53:04,569 INFO sqlalchemy.engine.base.Engine ()


[<User('haruair', 'Edward Kim', 'test1234')>,
 <User('wendy', 'Wendy Williams', 'foobar')>,
 <User('mary', 'Mary Contrary', 'xxg527')>,
 <User('fred', 'Fred Flinstone', 'blar')>]

In [36]:
# AND
# filter(and_(User.name == 'ed', User.fillname == 'Edward Kim'))
from sqlalchemy import and_
session.query(User).filter(and_(User.name == 'ed', User.fullname == 'Edward Kim')).all()

2018-06-21 17:53:04,577 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2018-06-21 17:53:04,578 INFO sqlalchemy.engine.base.Engine ('ed', 'Edward Kim')


[]

In [37]:
# AND
# 메소드 체인을 활용
session.query(User).filter(User.name == 'ed').filter(User.fullname == 'Edward Kim').all()

2018-06-21 17:53:04,586 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2018-06-21 17:53:04,588 INFO sqlalchemy.engine.base.Engine ('ed', 'Edward Kim')


[]

In [38]:
# OR
from sqlalchemy import or_
session.query(User).filter(or_(User.name == 'ed', User.name == 'wendy')).all()

2018-06-21 17:53:04,596 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? OR users.name = ?
2018-06-21 17:53:04,596 INFO sqlalchemy.engine.base.Engine ('ed', 'wendy')


[<User('wendy', 'Wendy Williams', 'foobar')>]

<br>
### **03 Query Filter 결과의 반환**
리스트와 Scalar 객체로 반환
1. all() : [리스트] 를 반환
1. Query()의 all(), one(), first() : 즉시 SQL 스칼라 객체를 반환

In [39]:
# .all() 
# 조건해당 객체들을 리스트로 출력
query = session.query(User) #.filter(User.name.like('%air')).order_by(User.id)
query.all()

2018-06-21 17:53:04,604 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2018-06-21 17:53:04,605 INFO sqlalchemy.engine.base.Engine ()


[<User('haruair', 'Edward Kim', 'test1234')>,
 <User('wendy', 'Wendy Williams', 'foobar')>,
 <User('mary', 'Mary Contrary', 'xxg527')>,
 <User('fred', 'Fred Flinstone', 'blar')>]

In [40]:
# 첫 튜플객체를 Scalar 로 출력
query.first()

2018-06-21 17:53:04,612 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
 LIMIT ? OFFSET ?
2018-06-21 17:53:04,613 INFO sqlalchemy.engine.base.Engine (1, 0)


<User('haruair', 'Edward Kim', 'test1234')>

<br>
## **5 SQL 에서 String 활용**
1. Query() 에서 문자열을 유연하게 쓸 수 있다. 
1. filter()와 order_by()에서 String을 쓸 수 있다.(SAWarning이 발생함에 유의)
1. params() 문자열로 구체적 내용을 적용시 활용
1. from_statement() 일반적 SQL 문법을 사용가능

<br>
### **01 String의 활용**
Query() 에서 문자열을 유연하게 쓸 수 있다.

In [41]:
# .filter('string')
for user in session.query(User).\
            filter("id<224").\
            order_by("id").all():
    print (user.name)

2018-06-21 17:53:04,623 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<224 ORDER BY users.id
2018-06-21 17:53:04,624 INFO sqlalchemy.engine.base.Engine ()
haruair
wendy
mary
fred


/home/markbaum/Python/django/lib/python3.6/site-packages/sqlalchemy/sql/elements.py:4267: SAWarning: Textual SQL expression 'id<224' should be explicitly declared as text('id<224') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


In [42]:
# .params() 세세한 문자열의 SQL을 활용
session.query(User).filter("id<:value and name=:name").\
    params(value=1234, name='fred').order_by(User.id).one()

2018-06-21 17:53:04,631 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE id<? and name=? ORDER BY users.id
2018-06-21 17:53:04,632 INFO sqlalchemy.engine.base.Engine (1234, 'fred')


/home/markbaum/Python/django/lib/python3.6/site-packages/sqlalchemy/sql/elements.py:4267: SAWarning: Textual SQL expression 'id<:value and name=:name' should be explicitly declared as text('id<:value and name=:name') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


<User('fred', 'Fred Flinstone', 'blar')>

In [43]:
# .from_statement()
session.query(User).from_statement(
                    "SELECT * FROM users WHERE name=:name").\
                    params(name='haruair').all()

2018-06-21 17:53:04,648 INFO sqlalchemy.engine.base.Engine SELECT * FROM users WHERE name=?
2018-06-21 17:53:04,648 INFO sqlalchemy.engine.base.Engine ('haruair',)


/home/markbaum/Python/django/lib/python3.6/site-packages/sqlalchemy/sql/elements.py:4267: SAWarning: Textual SQL expression 'SELECT * FROM users WHERE...' should be explicitly declared as text('SELECT * FROM users WHERE...') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


[<User('haruair', 'Edward Kim', 'test1234')>]

In [44]:
# .from_statement()
session.query("id", "name", "thenumber12").\
        from_statement("SELECT id, name, 12 as "
                "thenumber12 FROM users WHERE name=:name").\
        params(name='haruair').all()

2018-06-21 17:53:04,658 INFO sqlalchemy.engine.base.Engine SELECT id, name, 12 as thenumber12 FROM users WHERE name=?
2018-06-21 17:53:04,659 INFO sqlalchemy.engine.base.Engine ('haruair',)


/home/markbaum/Python/django/lib/python3.6/site-packages/sqlalchemy/sql/elements.py:4267: SAWarning: Textual SQL expression 'SELECT id, name, 12 as th...' should be explicitly declared as text('SELECT id, name, 12 as th...') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


[(1, 'haruair', 12)]

<br>
### **02 문자열 SQL의 장단점**
Query() 에서 문자열을 유연하게 쓸 수 있다.

In [45]:
q = session.query(User.id, User.name)
q.order_by("name").all()

2018-06-21 17:53:04,668 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users ORDER BY users.name
2018-06-21 17:53:04,669 INFO sqlalchemy.engine.base.Engine ()


[(4, 'fred'), (1, 'haruair'), (3, 'mary'), (2, 'wendy')]

In [46]:
from sqlalchemy import func

ua = aliased(User)
q  = q.from_self(User.id, User.name, ua.name).\
    filter(User.name < ua.name).\
    filter(func.length(ua.name) != func.length(User.name))
q.all()

2018-06-21 17:53:04,684 INFO sqlalchemy.engine.base.Engine SELECT anon_1.users_id AS anon_1_users_id, anon_1.users_name AS anon_1_users_name, users_1.name AS users_1_name 
FROM (SELECT users.id AS users_id, users.name AS users_name 
FROM users) AS anon_1, users AS users_1 
WHERE anon_1.users_name < users_1.name AND length(users_1.name) != length(anon_1.users_name)
2018-06-21 17:53:04,684 INFO sqlalchemy.engine.base.Engine ()


[(1, 'haruair', 'wendy'),
 (1, 'haruair', 'mary'),
 (3, 'mary', 'wendy'),
 (4, 'fred', 'haruair'),
 (4, 'fred', 'wendy')]

In [47]:
# aliased 된 User를 기준으로 정렬된다
q.order_by("name").all()

2018-06-21 17:53:04,693 INFO sqlalchemy.engine.base.Engine SELECT anon_1.users_id AS anon_1_users_id, anon_1.users_name AS anon_1_users_name, users_1.name AS users_1_name 
FROM (SELECT users.id AS users_id, users.name AS users_name 
FROM users) AS anon_1, users AS users_1 
WHERE anon_1.users_name < users_1.name AND length(users_1.name) != length(anon_1.users_name) ORDER BY users_1.name
2018-06-21 17:53:04,695 INFO sqlalchemy.engine.base.Engine ()


[(4, 'fred', 'haruair'),
 (1, 'haruair', 'mary'),
 (1, 'haruair', 'wendy'),
 (3, 'mary', 'wendy'),
 (4, 'fred', 'wendy')]

In [48]:
q.order_by(ua.name).all()

2018-06-21 17:53:04,704 INFO sqlalchemy.engine.base.Engine SELECT anon_1.users_id AS anon_1_users_id, anon_1.users_name AS anon_1_users_name, users_1.name AS users_1_name 
FROM (SELECT users.id AS users_id, users.name AS users_name 
FROM users) AS anon_1, users AS users_1 
WHERE anon_1.users_name < users_1.name AND length(users_1.name) != length(anon_1.users_name) ORDER BY users_1.name
2018-06-21 17:53:04,706 INFO sqlalchemy.engine.base.Engine ()


[(4, 'fred', 'haruair'),
 (1, 'haruair', 'mary'),
 (1, 'haruair', 'wendy'),
 (3, 'mary', 'wendy'),
 (4, 'fred', 'wendy')]

In [49]:
q.order_by(User.name).all()

2018-06-21 17:53:04,713 INFO sqlalchemy.engine.base.Engine SELECT anon_1.users_id AS anon_1_users_id, anon_1.users_name AS anon_1_users_name, users_1.name AS users_1_name 
FROM (SELECT users.id AS users_id, users.name AS users_name 
FROM users) AS anon_1, users AS users_1 
WHERE anon_1.users_name < users_1.name AND length(users_1.name) != length(anon_1.users_name) ORDER BY anon_1.users_name
2018-06-21 17:53:04,714 INFO sqlalchemy.engine.base.Engine ()


[(4, 'fred', 'haruair'),
 (4, 'fred', 'wendy'),
 (1, 'haruair', 'wendy'),
 (1, 'haruair', 'mary'),
 (3, 'mary', 'wendy')]

In [50]:
# 객체의 숫자 세기
session.query(User).filter(User.name.like('haru%')).count()

2018-06-21 17:53:04,726 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?) AS anon_1
2018-06-21 17:53:04,728 INFO sqlalchemy.engine.base.Engine ('haru%',)


1

In [51]:
from sqlalchemy import func
session.query(func.count(User.name), User.name).group_by(User.name).count()

2018-06-21 17:53:04,741 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT count(users.name) AS count_2, users.name AS users_name 
FROM users GROUP BY users.name) AS anon_1
2018-06-21 17:53:04,743 INFO sqlalchemy.engine.base.Engine ()


4

In [52]:
# User의 select_from 을 사용
session.query(func.count('*')).select_from(User).scalar()

2018-06-21 17:53:04,753 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM users
2018-06-21 17:53:04,755 INFO sqlalchemy.engine.base.Engine ('*',)


4

In [53]:
# User의 primary key를 사용하면 select_from 없이사용
session.query(func.count(User.id)).scalar() 

2018-06-21 17:53:04,766 INFO sqlalchemy.engine.base.Engine SELECT count(users.id) AS count_1 
FROM users
2018-06-21 17:53:04,768 INFO sqlalchemy.engine.base.Engine ()


4

<br>
### **03 관계(relationship) 만들기**
Query() 에서 문자열을 유연하게 쓸 수 있다.

In [54]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref

class Address(Base):
    __tablename__ = 'addresses'
    id            = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id       = Column(Integer, ForeignKey('users.id'))
    user          = relationship("User", backref=backref('addresses', order_by=id))

    def __init__(self, email_address):
        self.email_address = email_address

    def __repr__(self):
        return "<Address('%s')>" % self.email_address

In [55]:
class User(Base):

    __tablename__ = 'addresses'
    __table_args__ = {'extend_existing': True}  # 같은 이름의 User가 존재시 이를 덮어쓴다

    id            = Column(Integer, primary_key=True)
    addresses     = relationship("Address", order_by="Address.id", backref="user")
    email_address = Column(String, nullable=False)
    user_id       = Column(Integer, ForeignKey('users.id'))
    user          = relationship("User", backref=backref('addresses', order_by=id))

    def __init__(self, email_address):
        self.email_address = email_address
        
    def __repr__(self):
        return "<Address('%s')>" % self.email_address

/home/markbaum/Python/django/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  item.__name__


In [56]:
Base.metadata.create_all(engine)

2018-06-21 17:53:04,833 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-06-21 17:53:04,834 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,836 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-06-21 17:53:04,838 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,841 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2018-06-21 17:53:04,842 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 17:53:04,844 INFO sqlalchemy.engine.base.Engine COMMIT


<br>
## **6 Many To Many Relationship**
다대다 관계 

<br>
### **01 Blog 에서 다대다 관계(relationship)**
Query() 에서 문자열을 유연하게 쓸 수 있다.

In [57]:
# 조합 테이블
from sqlalchemy import Table, Text

post_keywords = Table('post_keywords', Base.metadata,
                   Column('post_id',    Integer, ForeignKey('posts.id')),
                   Column('keyword_id', Integer, ForeignKey('keywords.id')))

In [58]:
class BlogPost(Base):

    __tablename__ = 'posts'

    id       = Column(Integer, primary_key=True)
    user_id  = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body     = Column(Text)
    # 다대다 관계 : BlogPost <-> Keyword
    keywords = relationship('Keyword', secondary=post_keywords, backref='posts')

    def __init__(self, headline, body, author):
        self.author   = author
        self.headline = headline
        self.body     = body

    def __repr__(self):
        return "<BlogPost('%r', '%r', '%r')>" % (self.headline, self.body, self.author)

In [59]:
class Keyword(Base):
    __tablename__ = 'keywords'

    id      = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)

    def __init__(self,keyword):
        self.keyword = keyword

In [60]:
from sqlalchemy.orm import backref
# User에서의 관계를 "다이나믹" 로딩 처리
BlogPost.author = relationship(User, backref=backref('posts', lazy='dynamic'))

In [61]:
# 새로운 테이블을 정의한다
Base.meta.create_all(engine)

AttributeError: type object 'Base' has no attribute 'meta'

In [62]:
wendy = session.query(User).\
                filter_by(name='wendy').\
                one()
post = BlogPost("Wendy's Blog Post", "This is a test", wendy)
session.add(post)

NoForeignKeysError: Could not determine join condition between parent/child tables on relationship User.addresses - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.

In [63]:


post.keywords.append(Keyword('wendy'))
post.keywords.append(Keyword('firstpost')) 


NameError: name 'post' is not defined